# 12.2 网站分析

In [2]:
#!/usr/bin/python
# coding: utf-8

import requests
from bs4 import BeautifulSoup
import time

headers = {'User-Agent' : 'Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US; rv:1.9.1.6) Gecko/20091201 Firefox/3.5.6'} 
r = requests.get("https://en.wikipedia.org/wiki/Wikipedia", headers= headers)
html = r.text
bsObj = BeautifulSoup(html, "lxml")

for link in bsObj.findAll("a"):
    if 'href' in link.attrs:
        print (link.attrs['href'])

/wiki/Wikipedia:Protection_policy#semi
#mw-head
#p-search
/wiki/Main_Page
/wiki/Wikipedia:About
/wiki/Wikipedia_(disambiguation)
/wiki/Lists_of_encyclopedias
/wiki/File:Wikipedia-logo-v2.svg
/wiki/File:Wikipedia_wordmark.svg
/wiki/Wikipedia_logo
/wiki/Glyph
/wiki/Writing_system
/wiki/File:Wikipedia_Main_Page.png
/wiki/Main_Page
/wiki/Online_encyclopedia
/wiki/Wikimedia_Foundation
/wiki/Jimmy_Wales
/wiki/Larry_Sanger
#cite_note-Sidener-1
https://www.wikipedia.org
/wiki/Alexa_Internet
https://www.alexa.com/siteinfo/wikipedia.org
//en.wikipedia.org/w/index.php?title=Wikipedia&action=edit
#cite_note-2
/wiki/Registered_user
/wiki/List_of_Wikipedias#Grand_total
#cite_note-3
/wiki/List_of_Wikipedias#Grand_total
/wiki/Creative_Commons_licenses
/wiki/GFDL
/wiki/LAMP_(software_bundle)
#cite_note-roadchap-4
/wiki/OCLC
https://www.worldcat.org/oclc/52075003
/wiki/Help:IPA/English
/wiki/File:En-uk-Wikipedia.ogg
//upload.wikimedia.org/wikipedia/commons/c/c5/En-uk-Wikipedia.ogg
/wiki/Help:IPA/English

https://wayback.archive-it.org/all/20130308095103/http://www.bing.com/community/site_blogs/b/search/archive/2009/07/27/researching-with-bing-reference.aspxResearching
http://www.bing.com/community/site_blogs/b/search/archive/2009/07/27/researching-with-bing-reference.aspxResearching
#cite_ref-wikipediaondvd_authorized_1_273-0
http://www.wikipediaondvd.com/
https://web.archive.org/web/20130603205800/http://www.wikipediaondvd.com/
/wiki/Wayback_Machine
#cite_ref-wikipediaondvd_commercially_available_1_274-0
http://www.wikipediaondvd.com/site.php?temp=buy
https://web.archive.org/web/20130503073535/http://www.wikipediaondvd.com/site.php?temp=buy
/wiki/Wayback_Machine
#cite_ref-WM_polish_WP_on_dvd_275-0
http://meta.wikimedia.org/wiki/Polska_Wikipedia_na_DVD_%28z_Helionem%29/en
#cite_ref-WP_german_on_dvd_1_276-0
https://de.wikipedia.org/wiki/Wikipedia:DVD
#cite_ref-python.org_CDPedia_Argentina_1_277-0
http://python.org.ar/pyar/Proyectos/CDPedia
#cite_ref-WP_CD_selection_1_278-0
/wiki/Wikiped

# 12.3 项目实施（深度优先的递归爬虫）

In [2]:
#!/usr/bin/python
# coding: utf-8

import requests
import re
import time

exist_url = [] #存放已爬取的网页
g_writecount = 0

def scrappy(url, depth = 1):
    global g_writecount    
    try:
        headers = {'User-Agent' : 'Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US; rv:1.9.1.6) Gecko/20091201 Firefox/3.5.6'} 
        url = "https://en.wikipedia.org/wiki/" + url
        r = requests.get(url, headers= headers)
        html = r.text
    except Exception as e: 
        print ('Failed downloading and saving', url)
        print (e)
        exist_url.append(url)
        return None
    
    exist_url.append(url)
    link_list = re.findall('<a href="/wiki/([^:#=<>]*?)".*?</a>',html)
    # 去掉已爬链接和重复链接
    unique_list = list(set(link_list) - set(exist_url))
    
    #把所有链接写出到txt文件
    for eachone in unique_list:
        g_writecount += 1
        output = "No." + str(g_writecount) + "\t Depth:" +  str(depth) + "\t"+ url + ' -> ' + eachone + '\n'
        print (output)
        with open('link_12-3.txt', "a+") as f:
            f.write(output)

        #只获取两层，"Wikipedia"算第一层
        if depth < 2:
            #递归调用自己来访问下一层
            scrappy(eachone, depth+1)

scrappy("Wikipedia")

# 12.4 项目进阶（广度优先的多线程爬虫）


In [ ]:
#!/usr/bin/env python  
#coding=utf-8  
import threading  
import requests
import re  
import time  
g_mutex = threading.Condition()  
g_pages = [] #储存广度爬虫获取的html代码，之后解析所有url链接  
g_queueURL = [] #等待爬取的url链接列表  
g_existURL = [] #已经爬取过的url链接列表    
g_writecount = 0 #找到的链接数

class Crawler:  
    def __init__(self,url,threadnum):   
        self.url=url  
        self.threadnum=threadnum  
        self.threadpool=[]   

    def craw(self):  #爬虫的控制大脑，包括爬取网页，更新队列
        global g_queueURL  
        g_queueURL.append(url)
        depth=1
        while(depth < 3):    
            print ('Searching depth ',depth,'...\n')  
            self.downloadAll()  
            self.updateQueueURL()
            g_pages = []
            depth += 1  

    def downloadAll(self): #调用多线程爬虫，在小于线程最大值和没爬完队列之前，会增加线程
        global g_queueURL    
        i=0  
        while i<len(g_queueURL):  
            j=0  
            while j<self.threadnum and i+j < len(g_queueURL):    
                threadresult = self.download(g_queueURL[i+j],j)    
                j+=1  
            i += j  
            for thread in self.threadpool:  
                thread.join(30)  
            threadpool=[]  
        g_queueURL=[]  

    def download(self,url,tid): #调用多线程爬虫 
        crawthread=CrawlerThread(url,tid)  
        self.threadpool.append(crawthread)  
        crawthread.start()  

    def updateQueueURL(self): #完成一个深度的爬虫之后，更新队列
        global g_queueURL  
        global g_existURL  
        newUrlList=[]  
        for content in g_pages:  
            newUrlList+=self.getUrl(content)  
        g_queueURL=list(set(newUrlList)-set(g_existURL))    

    def getUrl(self,content): #从获取的网页中解析url 
        link_list = re.findall('<a href="/wiki/([^:#=<>]*?)".*?</a>',content)
        unique_list = list(set(link_list)) 
        return unique_list  

class CrawlerThread(threading.Thread): #爬虫线程
    def __init__(self,url,tid):  
        threading.Thread.__init__(self)  
        self.url=url  
        self.tid=tid  
    def run(self):  
        global g_mutex    
        global g_writecount  
        try:
            print (self.tid, "crawl ", self.url)
            headers = {'User-Agent' : 'Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US; rv:1.9.1.6) Gecko/20091201 Firefox/3.5.6'} 
            r = requests.get("https://en.wikipedia.org/wiki/" + self.url, headers= headers)
            html = r.text

            link_list2 = re.findall('<a href="/wiki/([^:#=<>]*?)".*?</a>',html)
            unique_list2 = list(set(link_list2))
            for eachone in unique_list2:
                g_writecount += 1
                content2 = "No." + str(g_writecount) + "\t Thread" +  str(self.tid) + "\t"+ self.url + '->' + eachone +'\n'
                with open('title2.txt', "a+") as f:
                    f.write(content2)
        except Exception as e: 
            g_mutex.acquire()  
            g_existURL.append(self.url)   
            g_mutex.release()  
            print ('Failed downloading and saving',self.url)
            print (e)
            return None
        g_mutex.acquire()  
        g_pages.append(html)  
        g_existURL.append(self.url)  
        g_mutex.release()

if __name__ == "__main__":
    url = "Wikipedia"
    threadnum = 5
    crawler = Crawler(url,threadnum)  
    crawler.craw()  